In [53]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
import requests
import json
import pprint
import time

import pandas as pd

# fix ssl certificate (needed for MacOS sometimes)
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [54]:
CAP_FRIENDLY_BASE_URL = 'https://www.capfriendly.com'

def get_soup(url):
    page = urlopen(url)
    html = page.read().decode("utf-8")
    soup = BeautifulSoup(html, 'html.parser')
    return soup

In [55]:
# # get links to each team page
# home_page_soup = get_soup(CAP_FRIENDLY_BASE_URL)

# teams = {}
# for team in home_page_soup.find('div', {'class': 'mb10'}).find_all('a', {'class': 'whi'}):
#     teams[team.get_text()] = team.get('href')
# pprint.pprint(teams)

In [56]:
# # scrape all staff members and get links to their pages
# staffs = {}
# for team, link in teams.items():
#     team_name_url = link.split('/')[2]
#     staff_link = f'{CAP_FRIENDLY_BASE_URL}/teams/staff/{team_name_url}'
#     print(staff_link)

#     staff_page_soup = get_soup(staff_link)
#     for staff in staff_page_soup.find('tbody').find_all('tr'):
#         for data in staff.find_all('a'):
#             staffs[data.get_text()] = data.get('href')

In [57]:
# len(staffs)

# General Managers page

In [58]:
url = 'https://www.capfriendly.com/general-managers'
gm_soup = get_soup(url)

gm_and_link = {}
for row in gm_soup.find('table', {'id': 'gm_table'}).find_all('tr')[1:]:
    name_data = row.find_all('td')[0]
#     gm_and_link.append({
#         'name': name_data.get_text(),
#         'link': name_data.find('a').get('href')
#     })
    gm_and_link[name_data.get_text()] = name_data.find('a').get('href')

In [43]:
gm_and_link['Eric Tulsky']

'/staff/eric-tulsky'

In [46]:
all_signings = []
for staff_member, link in gm_and_link.items():
    # scrape
    staff_member_soup = get_soup(CAP_FRIENDLY_BASE_URL + link)

    # extract data
    signigns_for_this_staff_member = []
    name = staff_member_soup.find('h1').get_text()
    try:
        for signing in staff_member_soup.find('div', {'id': 'staff_signings_area'}).find('tbody').find_all('tr'):
            signing_data = []
            for td in signing.find_all('td'):
                signing_data.append(td.get_text())
            for a in signing.find_all('a'):
                signing_data.append(a.get('href'))
            signing_data.append(name) # add GM name
            signigns_for_this_staff_member.append(signing_data)
            
        print(f'found {len(signigns_for_this_staff_member)} signings for staff member {name}')
        all_signings.extend(signigns_for_this_staff_member)
    except Exception:
        # not a gm, do nothing
        print(f'staff member {name} has zero signings as GM')
    time.sleep(0.25) # sleep to avoid api limits

found 15 signings for staff member JEFF JACKSON
found 12 signings for staff member ERIC TULSKY
found 194 signings for staff member DON WADDELL
found 103 signings for staff member BILL ARMSTRONG
found 20 signings for staff member STEVE STAIOS
found 189 signings for staff member KYLE DUBAS
found 31 signings for staff member BARRY TROTZ
found 241 signings for staff member BRAD TRELIVING
found 32 signings for staff member CRAIG CONROY
found 33 signings for staff member DANIEL BRIÈRE
found 65 signings for staff member CHRIS MACFARLAND
found 51 signings for staff member MIKE GRIER
found 62 signings for staff member KYLE DAVIDSON
found 58 signings for staff member PAT VERBEEK
found 68 signings for staff member PATRIK ALLVIN
found 54 signings for staff member KENT HUGHES
found 81 signings for staff member CHRIS DRURY
found 116 signings for staff member BILL ZITO
found 99 signings for staff member TOM FITZGERALD
found 111 signings for staff member KEVYN ADAMS
found 99 signings for staff member 

found 0 signings for staff member JOHN FERGUSON SR.
found 0 signings for staff member IRVING GRUNDMAN
found 0 signings for staff member FRED SHERO
found 0 signings for staff member LARRY GORDON
found 0 signings for staff member LOU NANNE
found 0 signings for staff member JAKE MILFORD
found 0 signings for staff member GEORGE MAGUIRE
found 0 signings for staff member JACK KELLEY
found 0 signings for staff member TED LINDSAY
found 0 signings for staff member BRIAN CONACHER
found 0 signings for staff member HARRY HOWELL
found 0 signings for staff member BAZ BASTIEN
found 0 signings for staff member RAY MIRON
found 0 signings for staff member BEP GUIDOLIN
found 0 signings for staff member BILL MCCREARY
found 0 signings for staff member RON RYAN
found 0 signings for staff member WREN BLAIR
found 0 signings for staff member SID SALOMON
found 0 signings for staff member RUDY PILOUS
found 0 signings for staff member ALEX DELVECCHIO
found 0 signings for staff member MAURICE FILION
found 0 signin

In [49]:
df = pd.DataFrame(all_signings)
df.columns = ['playerName', 'teamName', 'signDate', 'contractType', 'term', 'aav', 'totalValue', 'playerLink', 'teamLink', 'generalManager']
df.to_csv('contracts_tilJuly2024.csv', index=False)
df.head(10)

,playerName,teamName,signDate,contractType,term,aav,totalValue,playerLink,teamLink,generalManager
0,"Arvidsson, Viktor",Edmonton Oilers,"Jul. 1, 2024",Standard (UFA),2 years,"$4,000,000","$8,000,000",/players/viktor-arvidsson,/teams/oilers,JEFF JACKSON
1,"Skinner, Jeff",Edmonton Oilers,"Jul. 1, 2024",Standard (UFA),1 year,"$3,000,000","$3,000,000",/players/jeff-skinner,/teams/oilers,JEFF JACKSON
2,"Henrique, Adam",Edmonton Oilers,"Jul. 1, 2024",Standard (UFA),2 years,"$3,000,000","$6,000,000",/players/adam-henrique,/teams/oilers,JEFF JACKSON
3,"Janmark, Mattias",Edmonton Oilers,"Jul. 1, 2024",Standard (UFA),3 years,"$1,450,000","$4,350,000",/players/mattias-janmark,/teams/oilers,JEFF JACKSON
4,"Perry, Corey",Edmonton Oilers,"Jul. 1, 2024",35+ (UFA),1 year,"$1,150,000","$1,400,000",/players/corey-perry,/teams/oilers,JEFF JACKSON
5,"Pickard, Calvin",Edmonton Oilers,"Jun. 28, 2024",Standard (UFA),2 years,"$1,000,000","$2,000,000",/players/calvin-pickard,/teams/oilers,JEFF JACKSON
6,"Brown, Connor",Edmonton Oilers,"Jul. 1, 2024",Standard (UFA),1 year,"$1,000,000","$1,000,000",/players/connor-brown,/teams/oilers,JEFF JACKSON
7,"Brown, Joshua",Edmonton Oilers,"Jul. 1, 2024",Standard (UFA),3 years,"$1,000,000","$3,000,000",/players/joshua-brown,/teams/oilers,JEFF JACKSON
8,"Stecher, Troy",Edmonton Oilers,"Jul. 1, 2024",Standard (UFA),2 years,"$787,500","$1,575,000",/players/troy-stecher,/teams/oilers,JEFF JACKSON
9,"Dineen, Cam",Edmonton Oilers,"Jun. 28, 2024",Standard (UFA),2 years,"$775,000","$1,550,000",/players/cam-dineen,/teams/oilers,JEFF JACKSON


------------------

In [50]:
# scrape salary cap history
url = 'https://www.capfriendly.com/salary-cap'
salary_cap_soup = get_soup(url)

In [51]:
salary_cap_history = []
for tr in salary_cap_soup.find('div', {'id': 'salaryCapHistoryInnerContainer'}).find('tbody').find_all('tr'):
    this_year_salary_cap = []
    for td in tr.find_all('td'):
        this_year_salary_cap.append(td.get_text())
    salary_cap_history.append(this_year_salary_cap)

In [52]:
df_cap = pd.DataFrame(salary_cap_history)
df_cap.columns = ['season', 'dateConfirmed', 'percentChange', 'upperLimit', 'lowerLimit', 'minSalary']
df_cap.to_csv('capHistory.csv', index=False)
df_cap

,season,dateConfirmed,percentChange,upperLimit,lowerLimit,minSalary
0,2033-34,,-,"$92,000,000","$68,000,000","$775,000"
1,2032-33,,-,"$92,000,000","$68,000,000","$775,000"
2,2031-32,,-,"$92,000,000","$68,000,000","$775,000"
3,2030-31,,-,"$92,000,000","$68,000,000","$775,000"
4,2029-30,,-,"$92,000,000","$68,000,000","$775,000"
5,2028-29,,-,"$92,000,000","$68,000,000","$775,000"
6,2027-28,,-,"$92,000,000","$68,000,000","$775,000"
7,2026-27,,-,"$92,000,000","$68,000,000","$775,000"
8,2025-26,NHL Estimate,5.14%,"$92,000,000","$68,000,000","$775,000"
9,2024-25,"Jun. 8, 2024",5.39%,"$88,000,000","$65,000,000","$775,000"


# GM Careers

In [72]:
gm_data = []
for row in gm_soup.find('table', {'id': 'gm_table'}).find_all('tr')[1:]:
    this_gm = []
    for cell in row.find_all('td'):
        this_gm.append(cell.get_text())
        
    gm_data.append(this_gm)

In [81]:
df_gm = pd.DataFrame(gm_data)
df_gm.columns = ['name', 'team', 'dateSpan', 'ageSpan', 'duration', 'contracts', 'contractsValueUSD', 'drafted', 'trades']
df_gm.to_csv('generalManagers_tilJuly2024.csv', index=False)
df_gm

,name,team,dateSpan,ageSpan,duration,contracts,contractsValueUSD,drafted,trades
0,Chris Patrick,Washington Capitals,"Jul. 8, 2024 - Present",48 -,,-,-,-,-
1,Jeff Jackson (Interim),Edmonton Oilers,"Jun. 27, 2024 - Present",59 -,11 Days,16,"$37,300,000",7,2
2,Eric Tulsky,Carolina Hurricanes,"Jun. 18, 2024 - Present",-,20 Days,11,"$107,543,500",10,5
3,Don Waddell,Columbus Blue Jackets,"May 28, 2024 - Present",65 -,"1 Month, 10 Days",7,"$35,610,000",6,2
4,Bill Armstrong,Utah Hockey Club,"Apr. 18, 2024 - Present",53 -,"2 Months, 20 Days",19,"$63,525,000",11,4
...,...,...,...,...,...,...,...,...,...
326,Léo Dandurand,Montreal Canadiens,"Nov. 2, 1921 - Sep. 17, 1935",32 - 46,"13 Years, 10 Months, 15 Days",-,-,-,-
327,Charles Querrie,Toronto Maple Leafs,"Jan. 1917 - Feb. 14, 1927",39 - 49,10 Years,-,-,-,-
328,George Kennedy,Montreal Canadiens,"Jan. 1910 - Oct. 19, 1921",28 - 39,11 Years,-,-,-,-
329,Jack Laviolette,Montreal Canadiens,Dec. 1909 - Jan. 1910,30 - 30,,-,-,-,-
